In [1]:
import os, sys
from dask import compute, persist
from dask.distributed import Client, progress
from dask.diagnostics import ProgressBar
import dask.dataframe as dd
from dask_ml.preprocessing import StandardScaler
from dask_ml.model_selection import train_test_split #,KFold,RandomizedSearchCV, GridSearchCV #, RandomizedSearchCV
import matplotlib.pyplot as plt
import dask_xgboost as dxgb
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import KFold,RandomizedSearchCV , GridSearchCV
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score,matthews_corrcoef
import joblib
import numpy as np


In [3]:
# import dask_jobqueue
# from dask_jobqueue import SLURMCluster
# cluster = SLURMCluster(cores=32,
#                        processes=16,
#                        memory="128GB",
#                        walltime="24:00:00",
#                        queue="workq",
#                        interface="ipogif0",

#                        project="k1423"
#                       )
# print(cluster.job_script())
# cluster.scale(jobs=8)



In [4]:
# client = Client(cluster)
client = Client()

In [5]:
df = []
for x in os.listdir("data"):
    print (x)
    df_temp = dd.read_csv("data/%s"%(x) )  #,dtype=dtype)
    df.append(df_temp)

184B5.csv
BT483.csv
HCC1428.csv
HCC1806.csv
HCC202.csv
Hs578T.csv
MCF12A.csv
MDAMB231.csv
MDAMB468.csv
SKBR3.csv
UACC3199.csv
ZR751.csv


In [6]:
ddf = dd.concat(df,axis=0)

In [7]:
print ("filling NA")
# y = y.compute()
with ProgressBar():
    for m in ["p.HER2","p.PLCg2"] :
#         print (df[m])
        
        ddf["%s_c"%(m)] =ddf[m].fillna(ddf[m].mean() )#, inplace=True )

filling NA


In [8]:
ddf = dd.get_dummies(ddf.categorize()).persist()

In [10]:
ddf = ddf.drop(["p.HER2","p.PLCg2","cellID","fileID"],axis=1)

In [11]:
ddf.columns

Index(['time', 'b.CATENIN', 'cleavedCas', 'CyclinB', 'GAPDH', 'IdU', 'Ki.67',
       'p.4EBP1', 'p.Akt.Ser473.', 'p.AKT.Thr308.', 'p.AMPK', 'p.BTK',
       'p.CREB', 'p.ERK', 'p.FAK', 'p.GSK3b', 'p.H3', 'p.JNK', 'p.MAP2K3',
       'p.MAPKAPK2', 'p.MEK', 'p.MKK3.MKK6', 'p.MKK4', 'p.NFkB', 'p.p38',
       'p.p53', 'p.p90RSK', 'p.PDPK1', 'p.RB', 'p.S6', 'p.S6K', 'p.SMAD23',
       'p.SRC', 'p.STAT1', 'p.STAT3', 'p.STAT5', 'p.HER2_c', 'p.PLCg2_c',
       'treatment_EGF', 'treatment_full', 'treatment_iEGFR', 'treatment_iPI3K',
       'treatment_iPKC', 'treatment_iMEK', 'cell_line_184B5',
       'cell_line_BT483', 'cell_line_HCC1428', 'cell_line_HCC1806',
       'cell_line_HCC202', 'cell_line_Hs578T', 'cell_line_MCF12A',
       'cell_line_MDAMB231', 'cell_line_MDAMB468', 'cell_line_SKBR3',
       'cell_line_UACC3199', 'cell_line_ZR751'],
      dtype='object')

In [25]:
rounds = {}
genes= ['b.CATENIN','cleavedCas','CyclinB','GAPDH','IdU','Ki.67','p.4EBP1','p.Akt.Ser473.', 'p.AKT.Thr308.',
        'p.AMPK','p.BTK','p.CREB','p.ERK','p.FAK','p.GSK3b','p.H3','p.HER2_c','p.JNK','p.MAP2K3',
        'p.MAPKAPK2','p.MEK','p.MKK3.MKK6','p.MKK4','p.NFkB','p.p38','p.p53','p.p90RSK','p.PDPK1',
        'p.PLCg2_c','p.RB','p.S6','p.S6K','p.SMAD23','p.SRC','p.STAT1','p.STAT3','p.STAT5']

for x in range(len (genes)):
#     print (x)
    my_list = ['b.CATENIN','cleavedCas','CyclinB','GAPDH','IdU','Ki.67','p.4EBP1','p.Akt.Ser473.',
               'p.AKT.Thr308.','p.AMPK','p.BTK','p.CREB','p.ERK','p.FAK','p.GSK3b','p.H3','p.HER2_c','p.JNK',
               'p.MAP2K3','p.MAPKAPK2','p.MEK',   'p.MKK3.MKK6','p.MKK4','p.NFkB','p.p38','p.p53','p.p90RSK',
               'p.PDPK1','p.PLCg2_c','p.RB','p.S6','p.S6K','p.SMAD23','p.SRC','p.STAT1','p.STAT3','p.STAT5']

    if x+1 <len (genes):
#         my_list = genes 
#         print (x,genes[x], genes[x+1])
        my_list.pop(x)
#         my_list.pop(x)
#         my_list.pop(x+1)
#         print ( my_list )
#         rounds[x] = { "test":[genes[x], genes[x+1]] , "train":my_list}
        rounds[x] = { "test":[genes[x] ], "train":my_list}
#         y = df[[genes[x], genes[x+1]]]

In [26]:
# rounds[0]

{'test': ['b.CATENIN'],
 'train': ['cleavedCas',
  'CyclinB',
  'GAPDH',
  'IdU',
  'Ki.67',
  'p.4EBP1',
  'p.Akt.Ser473.',
  'p.AKT.Thr308.',
  'p.AMPK',
  'p.BTK',
  'p.CREB',
  'p.ERK',
  'p.FAK',
  'p.GSK3b',
  'p.H3',
  'p.HER2_c',
  'p.JNK',
  'p.MAP2K3',
  'p.MAPKAPK2',
  'p.MEK',
  'p.MKK3.MKK6',
  'p.MKK4',
  'p.NFkB',
  'p.p38',
  'p.p53',
  'p.p90RSK',
  'p.PDPK1',
  'p.PLCg2_c',
  'p.RB',
  'p.S6',
  'p.S6K',
  'p.SMAD23',
  'p.SRC',
  'p.STAT1',
  'p.STAT3',
  'p.STAT5']}

In [28]:
feats=  ['time','treatment_EGF', 'treatment_full', 'treatment_iEGFR', 'treatment_iPI3K','treatment_iPKC', 'treatment_iMEK']

for k in  rounds :
    for ff in feats: 
        rounds[k]["train"].append(ff)
        

In [34]:
random_grid = {
   'colsample_bytree':[0.4,0.6,0.8],
   'gamma':[0,0.03,0.1,0.3],
   'min_child_weight':[1.5,6,10],
   'learning_rate':[0.1,0.07],
   'max_depth':[3,5],
   'n_estimators':[1000,5000],
   'reg_alpha':[1e-5, 1e-2,  0.75],
   'reg_lambda':[1e-5, 1e-2, 0.45],
   'subsample':[0.6,0.95]  
}

TypeError: 'int' object is not iterable

In [ ]:
cv = KFold(2, shuffle=True)

In [35]:
# %%time
for k in  rounds :
    X = ddf[rounds[k]["train"]]
    y = ddf[rounds[k]["test"]]

    print ("model")
    model = dxgb.XGBRegressor(objective='reg:squarederror',verbosity=3,n_jobs=-1,n_estimators=100)
    
    X = X.to_dask_array(lengths=True)
    y = y.to_dask_array(lengths=True)

    print ("split")
    X_train, X_test, y_train, y_test = train_test_split(scaled_data,y,test_size=0.3, random_state=101)
    
        
    print ("scaling")
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    rf_random = RandomizedSearchCV(estimator = base_model, 
                               param_distributions = random_grid, 
                               n_iter = 1, cv = cv, 
                               verbose=2, 
                               random_state=101, n_jobs = -1)
    
    print ("training")
    with joblib.parallel_backend('dask'):
#         model.fit(X, y.flatten())
        rf_random.fit(X_train,y_train.flatten())
#     model.save_model('%s.model'%(rounds[k]))

    print ("##### best score ")
    rf_random.best_score_
    print ("##### best estimator ")
    rf_random.best_estimator_

    
    predictions = rf_random.predict( X_test)
    predictions = client.persist(predictions)
#     predictions
    
    
    
    print (k,rounds[k]["test"],"##################################")
    print ("R^2:",r2_score(y_test.compute(), predictions.compute()))
    print ("MAE:",mean_absolute_error(y_test.compute(), predictions.compute()))
    print ("MSE:",mean_squared_error(y_test.compute(), predictions.compute()))
    
    print()

scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
scaling
CPU times: user 17.3 s, sys: 1.58 s, total: 18.8 s
Wall time: 57.8 s
